# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

42

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(34514, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(30)

FPL_GW           16744
Min_points_30    16744
Clean_name       16744
Min_points_4     16744
Min_points_2     16744
Min_points       16744
Cmp%             15960
npxG             15954
SCA              15954
GCA              15954
Cmp              15954
Att              15954
Carries          15954
PrgP             15954
Blocks           15954
PrgC             15954
T_att            15954
T_succ           15954
xG               15954
xA               15954
Touches          15954
Tkl              15954
xP               15915
Min              15841
Pos              15841
Element_30       15840
Player_CS_30     15840
SoT_4            15840
CrdY_4           15840
CrdR_4           15840
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16744, 274)

In [14]:
X_test = df.copy()

In [15]:
df["xP_4"]

76       2.075000
77       0.666667
106      4.300000
107      4.375000
108      4.150000
           ...   
34509         NaN
34510         NaN
34511         NaN
34512         NaN
34513         NaN
Name: xP_4, Length: 16744, dtype: float64

In [16]:
file = open("models/GBR_positional_XGB.pkl",'rb')
model = pickle.load(file)

In [17]:
model

In [18]:
model.predict( X_test.head(5) )

[1.9840550619622936,
 2.1538304287454357,
 1.9570475970545125,
 1.6989414249697041,
 2.2891599093714072]

In [19]:
model.predict( X_test.head(3) )

[1.9840550619622936, 2.1538304287454357, 1.9570475970545125]

In [20]:
model.predict( X_test.head(4) )

[1.9840550619622936,
 2.1538304287454357,
 1.9570475970545125,
 1.6989414249697041]

In [21]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
107,107,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117318,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
108,108,2023-08-26,Sat,Home,Brentford,Crystal Palace,Aaron-Hickey,1.0,RB,90.0,...,0.69,1774.125598,1769.467049,NaN,1.466667,1.0,0.0,0.0,0.0,True


In [22]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True


# Predicting

In [23]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [24]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [25]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [26]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Ast_2,xG_2,xA_2,...,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,DEF,FWD,GK,MID
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
107,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
108,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34509,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34510,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34511,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34512,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
# X_test.shape,  model.predict(X_test).shape

In [28]:
%%time
preds = get_predictions(model, df, X_test)

16744 (16744, 274)
CPU times: total: 2.44 s
Wall time: 1.4 s


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [29]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
12105,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,12.5,8.837359
15802,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,7.9,7.595856
5514,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,7.403033
7784,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,MID,7.0,6.985061
11304,Martin Ødegaard,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,MID,8.5,6.835200
3104,Callum Wilson,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,FWD,7.9,6.778013
6106,Gabriel Martinelli Silva,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,MID,8.0,6.586280
6071,Gabriel Fernando de Jesus,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,FWD,7.9,6.454655
11966,Miguel Almirón Rejala,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,MID,6.4,6.384105
13363,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.6,6.156182


In [30]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
6316,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,5.644047
1498,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,GK,5.5,4.188415
5026,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,GK,5.5,3.849049
1008,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,GK,5.0,3.782761
12628,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,GK,5.5,3.602268
13989,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,GK,4.5,3.602223
8937,José Malheiro de Sá,4.0,2023-24,Wolves,Crystal Palace,0.0,1704.537109,1758.725586,GK,5.0,3.594324
1918,André Onana,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.755249,1927.032104,GK,5.0,3.384879
8763,Jordan Pickford,4.0,2023-24,Everton,Sheffield United,0.0,1691.463989,1631.764282,GK,4.5,3.310716
5374,Emiliano Martínez Romero,4.0,2023-24,Aston Villa,Liverpool,0.0,1825.741455,1946.521606,GK,5.0,3.284418


In [31]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
12121,Mohamed Salah,20.0,2023-24,Liverpool,Newcastle Utd,1.0,1946.521606,1884.512817,MID,12.5,8.837359
12115,Mohamed Salah,14.0,2023-24,Liverpool,Fulham,1.0,1946.521606,1732.639404,MID,12.5,8.837359
12134,Mohamed Salah,33.0,2023-24,Liverpool,Crystal Palace,1.0,1946.521606,1758.725586,MID,12.5,8.837359
12132,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1946.521606,1631.764282,MID,12.5,8.837359
12131,Mohamed Salah,30.0,2023-24,Liverpool,Brighton,1.0,1946.521606,1843.758179,MID,12.5,8.837359
12127,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1946.521606,1602.477417,MID,12.5,8.837359
12125,Mohamed Salah,24.0,2023-24,Liverpool,Burnley,1.0,1946.521606,1722.952881,MID,12.5,8.837359
12123,Mohamed Salah,22.0,2023-24,Liverpool,Chelsea,1.0,1946.521606,1781.043579,MID,12.5,8.837359
12139,Mohamed Salah,38.0,2023-24,Liverpool,Wolves,1.0,1946.521606,1704.537109,MID,12.5,8.837359
12119,Mohamed Salah,18.0,2023-24,Liverpool,Arsenal,1.0,1946.521606,1927.032104,MID,12.5,8.837359


In [32]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
315,Guglielmo Vicario,2.0,2023-24,Tottenham,Manchester Utd,1.0,1817.656738,1877.037720,GK,5.0,6.324765
314,Guglielmo Vicario,1.0,2023-24,Tottenham,Brentford,0.0,1815.873413,1824.330444,GK,5.0,6.022122
6330,Guglielmo Vicario,18.0,2023-24,Tottenham,Everton,1.0,1829.239014,1691.463989,GK,5.0,5.680781
6337,Guglielmo Vicario,25.0,2023-24,Tottenham,Wolves,1.0,1829.239014,1704.537109,GK,5.0,5.680781
6323,Guglielmo Vicario,11.0,2023-24,Tottenham,Chelsea,1.0,1829.239014,1781.043579,GK,5.0,5.680781
6332,Guglielmo Vicario,20.0,2023-24,Tottenham,Bournemouth,1.0,1829.239014,1660.290894,GK,5.0,5.680781
6342,Guglielmo Vicario,30.0,2023-24,Tottenham,Luton,1.0,1829.239014,1602.477417,GK,5.0,5.680781
6321,Guglielmo Vicario,9.0,2023-24,Tottenham,Fulham,1.0,1829.239014,1732.639404,GK,5.0,5.680781
6325,Guglielmo Vicario,13.0,2023-24,Tottenham,Aston Villa,1.0,1829.239014,1825.741455,GK,5.0,5.680781
6339,Guglielmo Vicario,27.0,2023-24,Tottenham,Crystal Palace,1.0,1829.239014,1758.725586,GK,5.0,5.680781


In [33]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,310.880471
Erling Haaland,266.431371
Trent Alexander-Arnold,264.516310
Bruno Borges Fernandes,251.378535
Ollie Watkins,243.601136
Martin Ødegaard,241.950972
Callum Wilson,236.987598
Gabriel Martinelli Silva,234.803238
Miguel Almirón Rejala,229.988024


# Saving to file

In [34]:
preds.to_csv("predictions/preds_next_season_xgb.csv", index=False)